In [1]:
#Import libraries
import pandas as pd
from itertools import combinations
import numpy as np

In [2]:
#import filtered (from UNIX) and cleande (in R) dataset
mydata = pd.read_csv('mydata.csv')
mydata

,Year,Title,Conference,Authors
0,2017,Public Transportation Mode Detection from Cell...,CIKM,"Guanyao Li,Chun-Jie Chen,Sheng-Yun Huang,Ai-Jo..."
1,2019,DSANet: Dual Self-Attention Network for Multiv...,CIKM,"Siteng Huang,Donglin Wang,Xuehan Wu,Ao Tang"
2,2020,Multi-task Adversarial Spatial-Temporal Networ...,CIKM,"Senzhang Wang,Hao Miao,Hao Chen,Zhiqiu Huang"
3,2020,A Framework for Analyzing the Impact of Missin...,CIKM,"Fabiola Santore,Eduardo Cunha de Almeida,Wagne..."
4,2020,Auxiliary-task Based Deep Reinforcement Learni...,CIKM,"Wei Shen 0005,Xiaonan He,Chuheng Zhang,Qiang N..."
...,...,...,...,...
8722,2020,GeoSiteSearch: A Tool to Map Vietnamese Diaspo...,ICWSM,"Madison G. Masten,Thien-Huong Ninh,Nicholas Tran"
8723,2020,Auditing Race and Gender Discrimination in Onl...,ICWSM,"Joshua Asplund,Motahhare Eslami,Hari Sundaram,..."
8724,2020,Driving the Last Mile: Characterizing and Unde...,ICWSM,"Hemank Lamba,Shashank Srikanth,Dheeraj Reddy P..."
8725,2018,Automatically Conceptualizing Social Media Ana...,ICWSM,"Soon-Gyo Jung,Joni O. Salminen,Jisun An,Haewoo..."


* ####  Construct a function that take a dataframe as input and returns a weighted edgelist. 

In [3]:
def df_transform(df):
    
    #Split authors with comma (,) and transform each row of column Authors to list 
    for i in range(len(df)):
        df.Authors.iloc[i] = list(str(df.Authors.iloc[i]).split(","))
    
    #For each row in the column Authors of the dataframe, find all the possible combinations and append it to a list
    #This list will represent the existing edges between authors.
    #We should mention that 'list edges' is a list of lists which contains tuples. 
    #Each tuple is a pair of authors that have co-authored a paper.
    edges=[]
    for k in list(df.Authors):
        a = list(combinations(k, 2))
        edges.append(a)
    
    #Append each tuple into a list.
    #So the new list contains all the pairs of authors that have co-authored a paper
    b = []
    for i in range(len(edges)):
        for j in range(len(edges[i])):
              b.append(edges[i][j])
    
    #define a new dataframe
    new_df = pd.DataFrame(b)
    new_df.columns = ['Author 1', 'Author 2']
    new_df['Weights'] = np.ones(len(b)) #Initializing column Weights with ones.
    
    #Calculate the Weights
    #If a row has Author 1: A and Author 2: B and another row has Author 1: B and Author 2: A (Symmetry)
    #then we will count their appearance with weight equal to 2 in one row and not 1 in each row.
    #In different words we take into account that a symmetric property exists between columns Author 1 and Author 2.

    final_df=(new_df.groupby(new_df[['Author 1', 'Author 2']].apply(lambda x: str(sorted(x)), axis=1))
              .agg({"Author 1": "first", "Author 2": "first", "Weights": "sum"}).reset_index(drop=True))

    return final_df

# 2016

In [4]:
mydata_2016 = mydata.loc[mydata.Year==2016]
mydata_2016

,Year,Title,Conference,Authors
5,2016,Separating-Plane Factorization Models: Scalabl...,CIKM,"Haolan Chen,Di Niu,Kunfeng Lai,Yu Xu,Masoud Ar..."
6,2016,Joint Collaborative Ranking with Social Relati...,CIKM,"Dimitrios Rafailidis,Fabio Crestani"
10,2016,Probabilistic Approaches to Controversy Detect...,CIKM,"Myungha Jang,John Foley,Shiri Dori-Hacohen,Jam..."
25,2016,Online Food Recipe Title Semantics: Combining ...,CIKM,"Tomasz Kusmierczyk,Kjetil Nørvåg"
28,2016,Bus Routes Design and Optimization via Taxi Da...,CIKM,"Seong-Ping Chuah,Huayu Wu 0001,Yu Lu 0003,Lian..."
...,...,...,...,...
8705,2016,Changing Names in Online News Comments at the ...,ICWSM,"Simranjit Singh Sachar,Nicholas Diakopoulos"
8707,2016,Automatic Summarization of Real World Events U...,ICWSM,"Nasser Alsaedi,Pete Burnap,Omer F. Rana"
8711,2016,Shirtless and Dangerous: Quantifying Linguisti...,ICWSM,"Ethan Fast,Tina Vachovsky,Michael S. Bernstein"
8715,2016,Understanding Anti-Vaccination Attitudes in So...,ICWSM,"Tanushree Mitra,Scott Counts,James W. Pennebaker"


In [5]:
authors_2016 = df_transform(mydata_2016)
authors_2016

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


,Author 1,Author 2,Weights
0,Charles M. Doutriaux,Alessandro D'Anca,1.0
1,Cosimo Palazzo,Alessandro D'Anca,1.0
2,Alessandro D'Anca,Davide Salomoni,1.0
3,Alessandro D'Anca,Dean N. Williams,1.0
4,Donatello Elia,Alessandro D'Anca,1.0
...,...,...,...
9661,Zhefeng Wang,Zijin Zhao,1.0
9662,Zhen Yue,Zhigeng Geng,1.0
9663,Zhi Wei,Zhengzhang Chen,1.0
9664,Zhenjie Zhang,Zhifeng Hao,1.0


In [6]:
#Create CSV for 2016
authors_2016.to_csv('authors_2016.csv', index = False)

# 2017

In [7]:
mydata_2017 = mydata.loc[mydata.Year==2017]
mydata_2017

,Year,Title,Conference,Authors
0,2017,Public Transportation Mode Detection from Cell...,CIKM,"Guanyao Li,Chun-Jie Chen,Sheng-Yun Huang,Ai-Jo..."
9,2017,A Temporal Attentional Model for Rumor Stance ...,CIKM,"Amir Pouran Ben Veyseh,Javid Ebrahimi,Dejing D..."
12,2017,BoostVHT: Boosting Distributed Streaming Decis...,CIKM,"Theodore Vasiloudis,Foteini Beligianni,Gianmar..."
14,2017,Exploiting User Consuming Behavior for Effecti...,CIKM,"Shen Liu,Hongyan Liu"
18,2017,Automatic Navbox Generation by Interpretable C...,CIKM,"Chenhao Xie 0002,Lihan Chen,Jiaqing Liang,Kezu..."
...,...,...,...,...
8695,2017,Data Sets: Word Embeddings Learned from Tweets...,ICWSM,"Quanzhi Li,Sameena Shah,Xiaomo Liu,Armineh Nou..."
8697,2017,Behavioral Analysis of Review Fraud: Linking M...,ICWSM,"Parisa Kaghazgaran,James Caverlee,Majid Alfifi"
8708,2017,Selfie-Presentation in Everyday Life: A Large-...,ICWSM,"Julia Deeb-Swihart,Christopher Polack,Eric Gil..."
8716,2017,Self-Disclosure and Channel Difference in Onli...,ICWSM,"Diyi Yang,Zheng Yao,Robert E. Kraut"


In [8]:
authors_2017 = df_transform(mydata_2017)
authors_2017

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


,Author 1,Author 2,Weights
0,Yinzhan Xu,Alex 'Sandy' Pentland,1.0
1,Yoshihiko Suhara,Alex 'Sandy' Pentland,1.0
2,Le Gruenwald,Laurent d'Orazio,1.0
3,Yali Wan,Neil O'Hare,1.0
4,Neil O'Hare,Yi Chang 0001,1.0
...,...,...,...
10903,Zhengzhang Chen,Zhichun Li,1.0
10904,Zhongyuan Li,Zhenyi Lin,1.0
10905,Zheqian Chen,Zhou Zhao,1.0
10906,Zhuoren Jiang,Zhi Tang,1.0


In [9]:
#Create CSV for 2017
authors_2017.to_csv('authors_2017.csv', index = False)

# 2018

In [10]:
mydata_2018 = mydata.loc[mydata.Year==2018]
mydata_2018

,Year,Title,Conference,Authors
7,2018,Deep Semantic Hashing with Multi-Adversarial T...,CIKM,"Bingning Wang,Kang Liu 0001,Jun Zhao 0001"
15,2018,Ready for Use: Subject-Independent Movement In...,CIKM,"Dalin Zhang,Lina Yao,Kaixuan Chen,Sen Wang 0001"
22,2018,Rumor Detection with Hierarchical Social Atten...,CIKM,"Han Guo,Juan Cao,Yazi Zhang,Junbo Guo,Jintao Li"
23,2018,Online Learning for Non-Stationary A/B Tests.,CIKM,"Andrés Muñoz Medina,Sergei Vassilvitskii,Dong Yin"
24,2018,Relevance Estimation with Multiple Information...,CIKM,"Junqi Zhang,Yiqun Liu 0001,Shaoping Ma,Qi Tian"
...,...,...,...,...
8700,2018,WhatApp Doc? A First Look at WhatsApp Public G...,ICWSM,"Kiran Garimella,Gareth Tyson"
8704,2018,#DebateNight: The Role and Influence of Social...,ICWSM,"Marian-Andrei Rizoiu,Timothy Graham 0001,Rui Z..."
8719,2018,Modeling Evolution of Topics in Large-Scale Te...,ICWSM,"Elaheh Momeni,Shanika Karunasekera,Palash Goya..."
8721,2018,Is This an STD? Please Help!: Online Informati...,ICWSM,"Alicia L. Nobles,Caitlin N. Dreisbach,Jessica ..."


In [11]:
authors_2018 = df_transform(mydata_2018)
authors_2018

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


,Author 1,Author 2,Weights
0,Cosimo Palazzo,Alessandro D'Anca,1.0
1,Alessandro D'Anca,Dean N. Williams,1.0
2,Donatello Elia,Alessandro D'Anca,1.0
3,Alessandro D'Anca,Fabrizio Antonio,1.0
4,Alessandro D'Anca,Giovanni Aloisio,1.0
...,...,...,...
12627,Zhong Chen 0003,Zhide Fang,1.0
12628,Zhifeng Bao,Zhiyong Peng,1.0
12629,Zhiting Hu,Zi Huang,1.0
12630,Ziwei Wang,Zi Huang,1.0


In [12]:
#Create CSV for 2018
authors_2018.to_csv('authors_2018.csv', index = False)

# 2019

In [13]:
mydata_2019 = mydata.loc[mydata.Year==2019]
mydata_2019

,Year,Title,Conference,Authors
1,2019,DSANet: Dual Self-Attention Network for Multiv...,CIKM,"Siteng Huang,Donglin Wang,Xuehan Wu,Ao Tang"
11,2019,Additive Explanations for Anomalies Detected f...,CIKM,"Ioana Giurgiu,Anika Schumann"
17,2019,Generative Question Refinement with Deep Reinf...,CIKM,"Ye Liu 0006,Chenwei Zhang,Xiaohui Yan,Yi Chang..."
20,2019,Privacy-Preserving Tensor Factorization for Co...,CIKM,"Jing Ma,Qiuchen Zhang,Jian Lou 0001,Joyce C. H..."
29,2019,Discovering Interesting Cycles in Directed Gra...,CIKM,"Florian Adriaens,Çigdem Aslay,Tijl De Bie,Aris..."
...,...,...,...,...
8688,2019,Using Platform Signals for Distinguishing Disc...,ICWSM,"Jack LaViolette,Bernie Hogan"
8709,2019,Measuring the Importance of User-Generated Con...,ICWSM,"Nicholas Vincent,Isaac L. Johnson,Patrick Shee..."
8712,2019,What Twitter Profile and Posted Images Reveal ...,ICWSM,"Sharath Chandra Guntuku,Daniel Preotiuc-Pietro..."
8714,2019,Demographic Differentials in Facebook Usage ar...,ICWSM,"Sofia Gil-Clavel,Emilio Zagheni"


In [14]:
authors_2019 = df_transform(mydata_2019)
authors_2019

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


,Author 1,Author 2,Weights
0,Austin Harris 0002,Amanda O'Grady,1.0
1,Jose Stovall,Amanda O'Grady,1.0
2,Amanda O'Grady,Mina Sartipi,1.0
3,Mahardhika Pratama,Choiru Za'in,1.0
4,Choiru Za'in,Weiping Ding 0001,1.0
...,...,...,...
18068,Zhongbao Zhang,Zichang Yin,1.0
18069,Zhongjie Wang 0002,Zubair Shafiq,1.0
18070,Zongtao Liu,Zhongyi Tang,1.0
18071,Zhou Zhou,Ziyu Guan,1.0


In [15]:
#Create CSV for 2019
authors_2019.to_csv('authors_2019.csv', index = False)

# 2020

In [16]:
mydata_2020 = mydata.loc[mydata.Year==2020]
mydata_2020

,Year,Title,Conference,Authors
2,2020,Multi-task Adversarial Spatial-Temporal Networ...,CIKM,"Senzhang Wang,Hao Miao,Hao Chen,Zhiqiu Huang"
3,2020,A Framework for Analyzing the Impact of Missin...,CIKM,"Fabiola Santore,Eduardo Cunha de Almeida,Wagne..."
4,2020,Auxiliary-task Based Deep Reinforcement Learni...,CIKM,"Wei Shen 0005,Xiaonan He,Chuheng Zhang,Qiang N..."
8,2020,Enhanced Story Representation by ConceptNet fo...,CIKM,"Shanshan Huang,Kenny Q. Zhu,Qianzi Liao,Libin ..."
13,2020,An Extensive Investigation of Machine Learning...,CIKM,"Jose F. Rodrigues,Jean Louis Pépin,Lorraine Go..."
...,...,...,...,...
8713,2020,Towards Quantifying the Distance between Opini...,ICWSM,"Saket Gurukar,Deepak Ajwani,Sourav Dutta 0001,..."
8718,2020,When Does Trust in Online Social Groups Grow?,ICWSM,"Shankar Iyer,Justin Cheng,Nick Brown,Xiuhua Wa..."
8722,2020,GeoSiteSearch: A Tool to Map Vietnamese Diaspo...,ICWSM,"Madison G. Masten,Thien-Huong Ninh,Nicholas Tran"
8723,2020,Auditing Race and Gender Discrimination in Onl...,ICWSM,"Joshua Asplund,Motahhare Eslami,Hari Sundaram,..."


In [17]:
authors_2020 = df_transform(mydata_2020)
authors_2020

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


,Author 1,Author 2,Weights
0,Brian O'Connor,Jibril Shittu,1.0
1,Brian O'Connor,Patrick McLeod,1.0
2,Brian O'Connor,Rich Anderson,1.0
3,Brian O'Connor,Shailesh Kulkarni,1.0
4,Diarmuid O'Reilly-Morgan,Elias Z. Tragos,1.0
...,...,...,...
18985,Zuobai Zhang,Zhongzhi Zhang,1.0
18986,Zuohui Fu,Zhou Qin 0001,1.0
18987,Ziqi Tan,Zhou Zhao,1.0
18988,Zhuang Liu 0001,Zhuang Liu 0005,1.0


In [18]:
#Create CSV for 2020
authors_2020.to_csv('authors_2020.csv', index = False)